## Вступление

Всем привет! На сегодняшнем семинаре мы познакомимся с библиотекой **PyTorch**. Он очень похож на Numpy, с одним лишь отличием (на самом деле их больше, но сейчас мы поговорим про самое главное) -- PyTorch может считать градиенты за вас. Таким образом вам не надо будет руками писать обратный проход в нейросетях.

#### Семинар построен следующим образом:

1. Вспоминаем Numpy и сравниваем операции в PyTorch
2. Создаем тензоры в PyTorch
3. Работаем с градиентами руками
4. Моя первая нейросеть 

### Вспоминаем Numpy и сравниваем операции в PyTorch

Мы можем создавать матрицы, перемножать их, складывать, транспонировать и в целом совершать любые матричные операции

In [ ]:
import numpy as np 
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

from sklearn.datasets import load_boston
from tqdm.notebook import tqdm

%matplotlib inline

In [ ]:
a = np.random.rand(5, 3) # создали случайную матрицу 
a

In [ ]:
print(f"Проверили размеры: {a.shape}")

In [ ]:
print(f"Добавили 5:\n{a + 5}")

In [ ]:
print(f"X*X^T:\n{a @ a.T}")

In [ ]:
print(f"Среднее по колонкам:\n{a.mean(axis=-1)}")

In [ ]:
print(f"Изменили размеры: {a.reshape(3, 5).shape}")

## Разминка.

При помощи numpy посчитайте сумму квадратов натуральных чисел от 1 до 10000.

In [ ]:
<YOUR CODE>

Аналогичные операции в **PyTorch** выглядят следующим образом, синтаксис почти не отличается:

In [ ]:
x = torch.rand(5, 3)
x

In [ ]:
print(f"Проверили размеры: {x.shape}")

In [ ]:
print(f"Добавили 5:\n{x + 5}")

In [ ]:
print(f"X*X^T:\n{x @ x.T}")

In [ ]:
print(f"Среднее по колонкам:\n{x.mean(dim=-1)}")

In [ ]:
print(f"Изменили размеры:\n{x.reshape([3, 5]).shape}")

Небольшой пример того, как меняются операции:

* `x.sum(axis=-1) -> x.sum(dim=-1)`
* `x.astype(np.int64) -> x.type(torch.int64)`

Для помощи вам есть [таблица](https://pytorch-for-numpy-users.wkentaro.com/), которая поможет вам найти аналог операции в Numpy.

### Создаем тензоры в PyTorch и снова изучаем базовые операции

In [ ]:
x = torch.empty(5, 3)  # пустой тензор (т.е. без инициализации)
print(x)

In [ ]:
x = torch.rand(5, 3)  # тензор со случайными числами
print(x)

In [ ]:
x = torch.zeros(5, 3, dtype=torch.int64)  # тензор с нулями и указанием типов чисел
print(x)

In [ ]:
x = torch.tensor([5.5, 3])  # конструируем тензор из питоновского листа
print(x)

In [ ]:
x = x.new_ones(5, 3, dtype=torch.float64)  # используем уже созданный тензор для создания тензора из единичек
print(x, x.shape)

In [ ]:
x = torch.randn_like(x, dtype=torch.float32)  # создаем матрицу с размерами как у x
print(x, x.shape)

In [ ]:
y = torch.rand(5, 3)
print(x + y)  # операция сложения

In [ ]:
z = torch.add(x, y)  # очередная операция сложения
print(z)

In [ ]:
torch.add(x, y, out=z)  # и наконец последний вид
print(z)

In [ ]:
z.zero_()  # зануление значений тензора
print(z)

In [ ]:
print(x * y)  # поэлементное умножение

In [ ]:
print(x.mm(y.T))  # матричное умножение

In [ ]:
print(x @ y.T)  # и опять матричное умножение

In [ ]:
print(x.unsqueeze(0).shape)  # добавили измерение в начало, аналог броадкастинга 

In [ ]:
print(x.unsqueeze(0).squeeze(0).shape)  # убрали измерение в начале, аналог броадкастинга 

Мы также можем делать обычные срезы и переводить матрицы назад в numpy:

In [ ]:
a = np.ones((3, 5))
x = torch.ones((3, 5))
print(np.allclose(x.numpy(), a))
print(np.allclose(x.numpy()[:, 1], a[:, 1]))

### Работаем с градиентами руками

In [ ]:
boston = load_boston()
plt.scatter(boston.data[:, -1], boston.target)

В pytorch есть возможность при создании тензора указывать нужно ли считать по нему градиент или нет, с помощью параметра `requires_grad`. Когда `requires_grad=True` мы сообщаем фреймворку, о том, что мы хотим следить за всеми тензорами, которые получаются из созданного. Иными словами, у любого тензора, у которого указан данный параметр, будет доступ к цепочке операций и преобразований совершенными с ними. Если эти функции дифференцируемые, то у тензора появляется параметр `.grad`, в котором хранится значение градиента.

Если к тензору, получающемуся в результате, применить метод `.backward()`, то фреймворк посчитает по цепочке градиенту для всех тензоров, у которых `requires_grad=True`.

In [ ]:
w = torch.rand(1, requires_grad=True)
b = torch.rand(1, requires_grad=True)

x = torch.tensor(boston.data[:, -1] / boston.data[:, -1].max(), dtype=torch.float32)
y = torch.tensor(boston.target, dtype=torch.float32)

assert w.grad is None # только создали тензоры и в них нет градиентов
assert b.grad is None

In [ ]:
y_pred = w * x + b # и опять совершаем операции с тензорами
loss = torch.mean((y_pred - y)**2) # совершаем операции с тензорами
loss.backward() # считаем градиенты

In [ ]:
assert w.grad is not None  # сделали операции и посчитали градиенты, значение должно было появиться
assert b.grad is not None

assert isinstance(w.grad, torch.Tensor)  # градиент — это тоже тензор
assert isinstance(b.grad, torch.Tensor)

print("dL/dw = \n", w.grad)
print("dL/db = \n", b.grad)

In [ ]:
from IPython.display import clear_output

num_iters = 100

for i in range(num_iters):
    y_pred = w * x + b
    # попробуйте сделать полиномиальную регрессию в данном предсказании и посчитать градиенты после
    loss = torch.mean((y_pred - y)**2)
    loss.backward()

    with torch.no_grad():
        # делаем шаг градиентного спуска с lr = .05
        w -= <YOUR CODE>
        b -= <YOUR CODE>

        # обнуляем градиенты, чтобы на следующем шаге опять посчитать и не аккумулировать их
        w.grad.zero_()
        b.grad.zero_()

    # рисуем картинки
    if (i + 1) % 5 == 0:
        clear_output(True)
        plt.scatter(x.numpy(), y.numpy())
        # PyTorch запрещает вызывать .numpy() на тензорах, у которых requires_grad=True, поэтому
        # вначале делаем копию тензора при помощи .detach()
        plt.scatter(x.numpy(), y_pred.detach().numpy(), color='orange', linewidth=5)
        plt.show()

        print(f"[Iteration {i}] loss = {loss.detach().numpy()}")
        if loss.detach().numpy() < 0.5:
            print("Done!")
            break

### Моя первая нейросеть

Для того, чтобы разобраться как обучать нейросети в PyTorch, нужно освоить три вещи: 

1. Как формировать батчи и пихать их в сетку
2. Как сделать сетку
3. Как написать цикл обучения

#### Как формировать батчи и пихать их в сетку

Чтобы в данном фреймворке иметь возможность итерироваться по данным и применять к ним преобразования, например, аугментации, о которых вы узнаете позже -- нужно создать свой класс унаследованный от `torch.utils.data.Dataset`.

Вот пример из документации:

```
class FaceLandmarksDataset(torch.utils.data.Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.landmarks_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:]
        landmarks = np.array([landmarks])
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}

        if self.transform:
            sample = self.transform(sample)

        return sample
```

Как вы видите, у такого класса должно быть два метода: 

* `__len__`: возвращает информацию о том, сколько объектов у нас в датасете
* `__getitem__`: возвращает семпл и таргет к нему


Теперь давайте напишем такой сами, в качестве датасета сгенерируем рандомные данные.

In [ ]:
class RandomDataset(torch.utils.data.Dataset):
    """Our random dataset"""
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return {'sample': torch.tensor(x[idx, :], dtype=torch.float), 'target': y[idx]}

In [ ]:
x = np.random.rand(1000, 5)
y = np.random.rand(1000)

In [ ]:
our_dataset = RandomDataset(x, y)

In [ ]:
our_dataset[1]  # [1] под капотом вызывает .__getitem__(1)

Для того, чтобы из данных получать батчи в pytorch используется такая сущность как даталоадер, который принимает на вход класс унаследованный от `torch.utils.data.Dataset`. Сейчас посмотрим на пример:

In [ ]:
dataloader = torch.utils.data.DataLoader(our_dataset, batch_size=4)

Работают с ним следующим образом:

In [ ]:
for batch in dataloader:
    batch_x = batch['sample']
    batch_y = batch['target']
    print('Sample:', batch_x)
    print('Target:', batch_y)

    break

#### Как сделать сетку

Для того, чтобы в high-level pytorch создавать нейросети используется модуль `nn`. Нейросеть должна быть унаследована от класса `nn.Module`. Пример как это может выглядеть:

```
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
       x = F.relu(self.conv1(x))
       return F.relu(self.conv2(x))
```

Как мы видим на данном примере, у данного класса должно быть метод `forward`, который определяет прямой проход нейросети. Также из класса выше видно, что модуль `nn` содержит в себе реализацию большинства слоев, а модуль `nn.functional` -- функций активаций.

Есть еще один способ создать нейросеть и давайте его разберем на практике:

In [ ]:
model = nn.Sequential(  # создаем контейнер, который инициализируем списком слоёв
    nn.Linear(5, 3),    # добавили слой с 5-ю нейронами на вход и 3-мя на выход
    nn.ReLU(),          # добавили функцию активации
    nn.Linear(3, 1),    # добавили слой с 3-мя нейронами на вход и 5-ю на выход
)

In [ ]:
y_pred = model(batch_x) # получили предсказания модели
y_pred

Обратите внимание на `grad_fn`. Тензор `y_pred` помнит про то, что последней операцией в его вычислительном графе была [`addmm`](https://pytorch.org/docs/stable/generated/torch.addmm.html), то есть (упрощая) `b + m @ x`. Это соответствует `nn.Linear` в конце модели!

#### Как написать цикл обучения
 
Давайте теперь соберем теперь загрузку данных, создание модели и обучим на уже созданном для нас датасете MNIST

In [ ]:
# Чтобы сайт, на котором выложен датасет, не принял нас за ботов, прикинемся браузером

import urllib

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [ ]:
from pathlib import Path
from torch.hub import _get_torch_home

# На Linux датасет скачается в ~/.cache/torch/datasets, но можете выбрать любую другую папку
mnist_path = Path(_get_torch_home()) / 'datasets'

mnist_train = torchvision.datasets.MNIST(
    mnist_path, train=True, download=True,
    transform=torchvision.transforms.ToTensor()
) # используем готовый класс от торча для загрузки данных для тренировки
mnist_valid = torchvision.datasets.MNIST(
    mnist_path, train=False, download=True,
    transform=torchvision.transforms.ToTensor()
) # используем готовый класс от торча для загрузки данных для валидации

train_dataloader = torch.utils.data.DataLoader(
    mnist_train, batch_size=4, shuffle=True, num_workers=1
) # так как это уже унаследованный от Dataset класс, его можно сразу пихать в даталоадер

valid_dataloader = torch.utils.data.DataLoader(
    mnist_valid, batch_size=4, shuffle=False, num_workers=1
) # так как это уже унаследованный от Dataset класс, его можно сразу пихать в даталоадер

In [ ]:
for i in [0, 1]:
    plt.subplot(1, 2, i + 1)
    plt.imshow(mnist_train[i][0].squeeze(0).numpy().reshape([28, 28]), cmap='gray')
    plt.title(str(mnist_train[i][1]))
plt.show()

In [ ]:
model = nn.Sequential(
    nn.Flatten(),         # превращаем картинку 28х28 в вектор размером 784
    nn.Linear(784, 128),  # входной слой размером 784 нейронов с выходом в 128 нейронов
    nn.ReLU(),            # функция активации релу
    nn.Linear(128, 10),   # ещё один линейный слой
)

opt = torch.optim.SGD(model.parameters(), lr=0.05) # создаем оптимизатор и передаем туда параметры модели

Посмотрите внимательно: мы сейчас будем заниматься классификацией, но в конце модели нет никакого софтмакса! Как так?

Дело в том, что поскольку во время обучения мы будем оптимизировать negative log likelihood, после softmax в функции потерь будет сразу стоять логарифм. Последовательное вычисление сначала softmax, а потом его логарифма может приводить к большим ошибкам округления, поэтому обычно эти две функции соединяют в композицию `log_softmax`, которая ведёт себя гораздо лучше:

$$
\log \left[ \operatorname{softmax}(x) \right]_i =
\log \left( \frac {\exp(x_i)} {\sum_{j=1}^n \exp(x_j)} \right) =
x_i - \log\left( \sum_{j=1}^n \exp(x_j) \right) =
x_i - \log\left( \sum_{j=1}^n \exp(x_j - x_* + x_*) \right) =
x_i - x_* - \log\left( \sum_{j=1}^n \exp(x_j - x_*) \right),
$$

где $x_* = \max \left\{ x_i \right\}$.

Поэтому мы можем:

* Либо поставить на выход модели функцию активации `log_softmax` и учить её с функцией потерь negative log likelihood (в PyTorch она называется `nll_loss`),
* Либо оставить модель безо всякой функции активации в конце и учить её с функцией потерь `nll_loss(log_softmax())`. В PyTorch такая композитная функция потерь называется `cross_entropy`, ей мы и воспользуемся.

А когда мы захотим предсказать классы, будет достаточно просто посчитать `argmax(model(), dim=-1).`

Веса моделей хранятся в виде матриц и выглядят так:

In [ ]:
[x for x in model.named_parameters()] 

In [ ]:
for epoch in range(1, 11):  # всего у нас будет 10 эпох (10 раз подряд пройдемся по всем батчам из трейна)
    # Трейн
    for x_train, y_train in tqdm(train_dataloader, desc=f'Epoch {epoch} | Train'):
        y_pred = model(x_train) # делаем предсказания
        loss = F.cross_entropy(y_pred, y_train) # считаем лосс
        
        ############################## Собственно обучение ##############################
        # 1. Обнуляем посчитанные градиенты параметров. Забыть про это — частая ошибка!
        opt.zero_grad()
        
        # 2. Считаем градиенты обратным проходом
        loss.backward()
        
        # 3. Обновляем параметры сети
        opt.step()
        #################################################################################

    # Валидация на каждой второй эпохе
    if epoch % 2 == 0:
        mean_valid_loss = [] # сюда будем складывать средний лосс по батчам
        valid_accuracy = []
        # мы считаем качество, поэтому мы запрещаем фреймворку считать градиенты по параметрам
        with torch.no_grad():
            for x_valid, y_valid in tqdm(valid_dataloader, desc=f'Epoch {epoch} | Valid'):
                y_pred = model(x_valid) # делаем предсказания
                loss = F.cross_entropy(y_pred, y_valid) # считаем лосс
                mean_valid_loss.append(loss.item()) # добавляем в массив
                valid_accuracy.extend((torch.argmax(y_pred, dim=-1) == y_valid).numpy().tolist())

        # выводим статистику
        print(f'Epoch: {epoch}, loss: {np.mean(mean_valid_loss):.5f}, accuracy: {np.mean(valid_accuracy)}')

### Дополнительные материалы:

* [PyTorch за 60 минут](http://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* [Использование PyTorch на GPU](https://pytorch.org/docs/master/notes/cuda.html)
* [Хорошая книга про PyTorch](https://pytorch.org/assets/deep-learning/Deep-Learning-with-PyTorch.pdf)

### Credits

Этот ноутбук основан на [ноутбуке](https://github.com/hse-ds/iad-deep-learning/blob/86313e3/sem01/sem01.ipynb) первого семинара курса по ИДА в Вышке, который, в свою очередь, основан на вводном [ноутбуке](https://github.com/yandexdataschool/Practical_DL/blob/fall20/week02_autodiff/seminar_pytorch.ipynb) второй недели курса по Deep Learning в ШАДе.